In [ ]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_00_load_data")' FastaiNotebook_00_load_data

Installing packages:
	.package(path: "/home/clattner/fastai_docs/dev_swift/FastaiNotebook_00_load_data")
		FastaiNotebook_00_load_data
With SwiftPM flags: []
Working in: /tmp/tmp0u49m3ue
/home/clattner/swift/usr/bin/swift-build: /home/clattner/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/clattner/swift/usr/lib/swift/linux/libFoundation.so)
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 1.96s
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
/home/clattner/swift/usr/bin/swiftc: /home/clattner/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/clattner/swift/usr/bin/swiftc)
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'Just' (1 sources)
/home/clattner/swift/usr/bin/swiftc: /home/clattner

: 

In [ ]:
//export
import Path
import TensorFlow

In [ ]:
import FastaiNotebook_00_load_data

## Get some Tensors to play with

In [ ]:
let xTrain = Tensor<Float>(randomNormal: [5, 784])
//let (xTrain, yTrain, xValid, yValid) = loadMNIST(path: mnistPath, flat: true)
var weights = Tensor<Float>(randomNormal: [784, 10]) / sqrt(784)
print(weights[0])

[ 0.033045642,   0.04449675,  0.015506563,   0.02996331, -0.039954513,  0.086654566,
  0.007328817, 0.0069880965,  0.022950158, -0.014337598]


# Building Matmul

Ok, now that we know how floating point types and arrays work, we can finally build our own matmul from scratch, using a few loops.  We will take the two input matrices as single dimentional arrays so we can show manual indexing into them, the hard way:

In [ ]:
// a and b are the flattened array elements, aDims/bDims are the #rows/columns of the arrays.
func swiftMatmul(a: [Float], b: [Float], aDims: (Int,Int), bDims: (Int,Int)) -> [Float] {
    assert(aDims.1 == bDims.0, "matmul shape mismatch")
    
    var res = Array(repeating: Float(0.0), count: aDims.0 * bDims.1)
    for i in 0 ..< aDims.0 {
        for j in 0 ..< bDims.1 {
            for k in 0 ..< aDims.1 {
                res[i+aDims.0*j] += a[i+aDims.0*k] * b[k+bDims.0*j]
            }
        }
    }
    return res
}

In [ ]:
// To try it out, we extract the scalars out of our MNIST data as an array.
let flatA = xTrain[0..<5].scalars
let flatB = weights.scalars
let (aDims,bDims) = ((5, 784), (784, 10))

In [ ]:
// Now that we've got everything together, we can try it out!
var resultArray = swiftMatmul(a: flatA, b: flatB, aDims: aDims, bDims: bDims)

In [ ]:
time(repeating: 100) {
    _ = swiftMatmul(a: flatA, b: flatB, aDims: aDims, bDims: bDims)
}

average: 0.12145337000000003 ms,   min: 0.110486 ms,   max: 0.153928 ms


Awesome, that is pretty fast - compare that to **835 ms** with Python!

You might be wondering what that `time(repeating:)` builtin is.  As you might guess, this is actually a Swift function - one that is using "trailing closure" syntax to specify the body of the timing block.  Trailing closures are passed as arguments to the function, and in this case, the function was defined in our ✅**00_load_data** workbook.  Let's take a look!


# Getting the performance of C 💯

This performance is pretty great, but we can do better.  Swift is a memory safe language (like Python), which means it has to do array bounds checks and some other stuff.  Fortunately, Swift is a pragmatic language that allows you to drop through this to get peak performance - check out Jeremy's article [High Performance Numeric Programming with Swift: Explorations and Reflections](https://www.fast.ai/2019/01/10/swift-numerics/) for a deep dive.

One thing you can do is use `UnsafePointer` (which is basically a raw C pointer) instead of using a bounds checked array.  This isn't memory safe, but gives us about a 2x speedup in this case!


In [ ]:
// a and b are the flattened array elements, aDims/bDims are the #rows/columns of the arrays.
func swiftMatmulUnsafe(a: [Float], b: [Float], aDims: (Int,Int), bDims: (Int,Int)) -> [Float] {
    assert(aDims.1 == bDims.0, "matmul shape mismatch")
    
    var res = Array(repeating: Float(0.0), count: aDims.0 * bDims.1)
    res.withUnsafeMutableBufferPointer { res in 
        for i in 0 ..< aDims.0 {
            for j in 0 ..< bDims.1 {
                for k in 0 ..< aDims.1 {
                    res[i+aDims.0*j] += a[i+aDims.0*k] * b[k+bDims.0*j]
                }
            }
        }
    }
    return res
}
time(repeating: 100) {
    _ = swiftMatmulUnsafe(a: flatA, b: flatB, aDims: aDims, bDims: bDims)
}

average: 0.07645685999999997 ms,   min: 0.073942 ms,   max: 0.105068 ms


One of the other cool things about this is that we can provide a nice idiomatic API to the caller of this, and keep all the unsafe shenanigans inside the implementation of this function.

If you really want to fall down the rabbit hole, you can look at the [implementation of `UnsafePointer`](https://github.com/apple/swift/blob/tensorflow/stdlib/public/core/UnsafePointer.swift), which is of written in Swift wrapping LLVM pointer operations.  This means you can literally get the performance of C code directly in Swift, while providing easy to use high level APIs!

## Swift 💖 C APIs too: you get the full utility of the C ecosystem

Swift even lets you transparently work with C APIs, just like it does with Python.  This can be used for both good and evil.  For example, here we directly call the `malloc` function, dereference the uninitialized pointer, and print it out:


In [ ]:
import Glibc

let ptr : UnsafeMutableRawPointer = malloc(42)

print("☠️☠️ Uninitialized garbage =", ptr.load(as: UInt8.self))

free(ptr)

☠️☠️ Uninitialized garbage = 192


An `UnsafeMutableRawPointer` ([implementation](https://github.com/apple/swift/blob/tensorflow/stdlib/public/core/UnsafeRawPointer.swift)) isn't something you should use lightly, but when you work with C APIs, you'll see various types like that in the function signatures.

Calling `malloc` and `free` directly aren't recommended in Swift, but is useful and important when you're working with C APIs that expect to get malloc'd memory, which comes up when you're written a safe Swift wrapper for some existing code.

Speaking of existing code, let's take a look at that **Python interop** we touched on before:


In [ ]:
import Python
let np = Python.import("numpy")
let pickle = Python.import("pickle")
let sys = Python.import("sys")

print("🐍list =    ", pickle.dumps([1, 2, 3]))
print("🐍ndarray = ", pickle.dumps(np.array([[1, 2], [3, 4]])))


🐍list =     b'\x80\x03]q\x00(K\x01K\x02K\x03e.'
🐍ndarray =  b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\x02K\x02\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i8q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89C \x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00q\rtq\x0eb.'


Of course this is [all written in Swift](https://github.com/apple/swift/tree/tensorflow/stdlib/public/Python) as well.  You can probably guess how this works now: `PythonObject` is a Swift struct that wraps a pointer to the Python interpreter's notion of a Python object.

```swift
@dynamicCallable
@dynamicMemberLookup
public struct PythonObject {
  var reference: PyReference
  ...
}
```

The `@dynamicMemberLookup` attribute allows it to dynamically handle all member lookups (like `x.y`) by calling into [the `PyObject_GetAttrString` runtime call](https://github.com/apple/swift/blob/tensorflow/stdlib/public/Python/Python.swift#L427).  Similarly, the `@dynamicCallable` attribute allows the type to intercept all calls to a PythonObject (like `x()`), which it implements using the [`PyObject_Call` runtime call](https://github.com/apple/swift/blob/tensorflow/stdlib/public/Python/Python.swift#L324).  

Because Swift has such simple and transparent access to C, it allows building very nice first-class Swift APIs that talk directly to the lower level implementation, and these implementations can have very little overhead.

# Working with Tensor

Lets get back into matmul and explore more of the `Tensor` type as provided by the TensorFlow module.  You can see all things `Tensor` can do in the [official documentation](https://www.tensorflow.org/swift/api_docs/Structs/Tensor).

Here are some highlights: You can create Tensor's in many ways using the initializer, for example you can get zeros or random data:


In [ ]:
var bias = Tensor<Float>(zeros: [10])

let m1 = Tensor<Float>(randomNormal: [5, 784])
let m2 = Tensor<Float>(randomNormal: [784, 10])

// Tensors carry data and a shape.
print("m1: ", m1.shape)
print("m2: ", m2.shape)

m1:  TensorShape(dimensions: [5, 784])
m2:  TensorShape(dimensions: [784, 10])


The `Tensor` type provides all the normal stuff you'd expect as methods.  Including arithemic, convolutions, etc and this includes full support for broadcasting:


In [ ]:
let small = Tensor<Float>([[1, 2],
                           [3, 4]])

print("🔢2x2:\n",             small)
print("\n⊞ matmul:\n",        matmul(small, small))
print("\n➕add:\n",           small + small)
print("\n💠➕broadcast add:\n", small + 100)

🔢2x2:
 [[1.0, 2.0],
 [3.0, 4.0]]

⊞ matmul:
 [[ 7.0, 10.0],
 [15.0, 22.0]]

➕add:
 [[2.0, 4.0],
 [6.0, 8.0]]

💠➕broadcast add:
 [[101.0, 102.0],
 [103.0, 104.0]]


**MatMul Operator:** In addition to using the global `matmul(a, b)` function, you can also use the `a • b` operator to matmul together two things.  This is just like the `@` operator in Python.  You can get it with the <kbd>option</kbd>-<kbd>8</kbd> on Mac or <kbd>compose</kbd>-<kbd>.</kbd>-<kbd>=</kbd> elsewhere. Or if you prefer, just use the `matmul()` function we've seen already.

In [ ]:
print("⊞ matmul:\n",        matmul(small, small))
print("\n⊞ again:\n",        small • small)


⊞ matmul:
 [[ 7.0, 10.0],
 [15.0, 22.0]]

⊞ again:
 [[ 7.0, 10.0],
 [15.0, 22.0]]


**Reshaping** works the way you'd expect:

In [ ]:
var m = Tensor([1.0, 2, 3, 4, 5, 6, 7, 8, 9]).reshaped(to: [3, 3])
print(m)
print()

print(sqrt((m * m).sum()))

[[1.0, 2.0, 3.0],
 [4.0, 5.0, 6.0],
 [7.0, 8.0, 9.0]]

16.881943016134134


## Elementwise ops and comparisons

Standard math operators (`+`,`-`,`*`,`/`) are all element-wise, and there are a bunch of standard math functions like `sqrt` and `pow`.  Here are some examples:

In [ ]:
var a = Tensor([10.0, 6, -4])
var b = Tensor([2.0, 8, 7])
(a,b)

▿ 2 elements
  - .0 : [10.0,  6.0, -4.0]
  - .1 : [2.0, 8.0, 7.0]


In [ ]:
print("add:  ", a + b)
print("mul:  ", a * b)
print("sqrt: ", sqrt(a))
print("pow:  ", pow(a, b))

add:   [12.0, 14.0,  3.0]
mul:   [ 20.0,  48.0, -28.0]
sqrt:  [3.1622776601683795,  2.449489742783178,               -nan]
pow:   [             100.0, 1679616.0000000002,           -16384.0]



**Comparison operators** (`>`,`<`,`==`,`!=`,...) in Swift are supposed to return a single `Bool` value, so they are `true` if all the elements of the tensors satisfy the comparison.

Elementwise versions have the `.` prefix, which is read as "pointwise": `.>`, `.<`, `.==`, etc.  You can merge a tensor of bools into a single Bool with the `any()` and `all()` methods.

In [ ]:
a < b

false


In [ ]:
a .< b

[false,  true,  true]


In [ ]:
print(a .> 0)

[ true,  true, false]


In [ ]:
print((a .> 0).all())

false


In [ ]:
print((a .> 0).any())

true


## Broadcasting

Broadcasting with a scalar works just like in Python:

In [ ]:
var a = Tensor([10.0, 6.0, -4.0])

In [ ]:
print(a+1)

[11.0,  7.0, -3.0]


In [ ]:
2 * m

[[ 2.0,  4.0,  6.0],
 [ 8.0, 10.0, 12.0],
 [14.0, 16.0, 18.0]]


#### Broadcasting a vector with a matrix

In [ ]:
let c = Tensor([10.0,20.0,30.0])

By default, broadcasting is done by adding 1 dimensions to the beginning until dimensions of both objects match.

In [ ]:
m + c

[[11.0, 22.0, 33.0],
 [14.0, 25.0, 36.0],
 [17.0, 28.0, 39.0]]


In [ ]:
c + m

[[11.0, 22.0, 33.0],
 [14.0, 25.0, 36.0],
 [17.0, 28.0, 39.0]]


To broadcast on the other dimensions, one has to use `expandingShape` to add the dimension.

In [ ]:
m + c.expandingShape(at: 1)

[[11.0, 12.0, 13.0],
 [24.0, 25.0, 26.0],
 [37.0, 38.0, 39.0]]


In [ ]:
c.expandingShape(at: 1)

[[10.0],
 [20.0],
 [30.0]]


#### Broadcasting rules

In [ ]:
print(c.expandingShape(at: 0).shape)

TensorShape(dimensions: [1, 3])


In [ ]:
print(c.expandingShape(at: 1).shape)

TensorShape(dimensions: [3, 1])


In [ ]:
c.expandingShape(at: 0) * c.expandingShape(at: 1)

[[100.0, 200.0, 300.0],
 [200.0, 400.0, 600.0],
 [300.0, 600.0, 900.0]]


In [ ]:
c.expandingShape(at: 0) .> c.expandingShape(at: 1)

[[false,  true,  true],
 [false, false,  true],
 [false, false, false]]


# Matmul using `Tensor`

Coming back to our matmul algorithm, we can implement exactly what we had before by using subscripting into a tensor, instead of subscripting into an array.  Let's see how that works:

In [ ]:
func tensorMatmul(_ a: Tensor<Float>, _ b: Tensor<Float>) -> Tensor<Float> {
    var res = Tensor<Float>(zeros: [a.shape[0], b.shape[1]])

    for i in 0 ..< a.shape[0] {
        for j in 0 ..< b.shape[1] {
            for k in 0 ..< a.shape[1] {
                res[i, j] += a[i, k] * b[k, j]
            }
        }
    }
    return res
}

In [ ]:
time { 
    let tmp = tensorMatmul(m1, m2)
    
    // Copy a scalar back to the host to force a GPU sync.
    _ = tmp[0, 0].scalar
}

average: 6805.451182 ms,   min: 6805.451182 ms,   max: 6805.451182 ms


What, what just happened?? We used to be less than a **tenth of a millisecond**, now we're taking **multiple seconds**.  It turns out that Tensor's are very good at bulk data processing, but they are not good at doing one float at a time.  Make sure to use the coarse-grained operations.  We can make this faster by vectorizing each loop in turn.

## Vectorize the inner loop into a multiply + sum

In [ ]:
func elementWiseMatmul(_ a:Tensor<Float>, _ b:Tensor<Float>) -> Tensor<Float>{
    let (ar, ac) = (a.shape[0], a.shape[1])
    let (br, bc) = (b.shape[0], b.shape[1])
    var res = Tensor<Float>(zeros: [ac, br])
    
    for i in 0 ..< ar {
        let row = a[i]
        for j in 0 ..< bc {
            res[i, j] = (row * b.slice(lowerBounds: [0,j], upperBounds: [ac,j+1]).squeezingShape(at: 1)).sum()
        }
    }
    return res
}

In [ ]:
_ = elementWiseMatmul(m1, m2)

In [ ]:
time { 
    let tmp = elementWiseMatmul(m1, m2)

    // Copy a scalar back to the host to force a GPU sync.
    _ = tmp[0, 0].scalar
}

average: 374.769131 ms,   min: 374.769131 ms,   max: 374.769131 ms


## Vectorize the inner two loops with broadcasting

In [ ]:
func broadcastMatmult(_ a:Tensor<Float>, _ b:Tensor<Float>) -> Tensor<Float>{
    var res = Tensor<Float>(zeros: [a.shape[0], b.shape[1]])
    for i in 0..<a.shape[0] {
        res[i] = (a[i].expandingShape(at: 1) * b).sum(squeezingAxes: 0)
    }
    return res
}

In [ ]:
_ = broadcastMatmult(m1, m2)

In [ ]:
time(repeating: 100) {
    let tmp = broadcastMatmult(m1, m2)

    // Copy a scalar back to the host to force a GPU sync.
    _ = tmp[0, 0].scalar
}

average: 1.8719801600000006 ms,   min: 1.477153 ms,   max: 2.784409 ms


## Vectorize the whole thing with one Tensorflow op

In [ ]:
time(repeating: 100) { _ = m1 • m2 }

average: 0.01714434 ms,   min: 0.013983 ms,   max: 0.033012 ms


The reason that TensorFlow works in practice is that it can scale way up to large matrices, for example, lets try some thing a bit larger:

In [ ]:
func timeMatmulTensor(size: Int) {
    var matrix = Tensor<Float>(randomNormal: [size, size])
    print("\n\(size)x\(size):\n  ⏰", terminator: "")
    time(repeating: 10) { 
        let matrix = matrix • matrix 
        _ = matrix[0, 0].scalar
    }
}

timeMatmulTensor(size: 1)     // Tiny
timeMatmulTensor(size: 10)    // Bigger
timeMatmulTensor(size: 100)   // Even Bigger
timeMatmulTensor(size: 1000)  // Biggerest
timeMatmulTensor(size: 5000)  // Even Biggerest


1x1:
  ⏰average: 0.19747569999999998 ms,   min: 0.134466 ms,   max: 0.406267 ms

10x10:
  ⏰average: 0.16372099999999998 ms,   min: 0.148772 ms,   max: 0.204141 ms

100x100:
  ⏰average: 0.1633207 ms,   min: 0.144609 ms,   max: 0.192945 ms

1000x1000:
  ⏰average: 0.49224870000000004 ms,   min: 0.475513 ms,   max: 0.517417 ms

5000x5000:
  ⏰average: 29.394594 ms,   min: 29.305466 ms,   max: 29.509257 ms


In constrast, our simple CPU implementation takes a lot longer to do the same work.  For example:

In [ ]:
func timeMatmulSwift(size: Int, repetitions: Int = 10) {
    var matrix = Tensor<Float>(randomNormal: [size, size])
    let matrixFlatArray = matrix.scalars

    print("\n\(size)x\(size):\n  ⏰", terminator: "")
    time(repeating: repetitions) { 
       _ = swiftMatmulUnsafe(a: matrixFlatArray, b: matrixFlatArray, aDims: (size,size), bDims: (size,size))
    }
}

timeMatmulSwift(size: 1)     // Tiny
timeMatmulSwift(size: 10)    // Bigger
timeMatmulSwift(size: 100)   // Even Bigger
timeMatmulSwift(size: 1000, repetitions: 1)  // Biggerest

print("\n5000x5000: skipped, it takes tooo long!")


1x1:
  ⏰average: 0.00022530000000000003 ms,   min: 0.000202 ms,   max: 0.000286 ms

10x10:
  ⏰average: 0.0020340000000000002 ms,   min: 0.002015 ms,   max: 0.002108 ms

100x100:
  ⏰average: 1.7244304000000004 ms,   min: 1.67428 ms,   max: 1.829252 ms

1000x1000:
  ⏰average: 2287.614728 ms,   min: 2287.614728 ms,   max: 2287.614728 ms

5000x5000: skipped, it takes tooo long!


Why is TensorFlow *so so so* much faster than our CPU implementation?  Well there are two reasons: the first of which is that it uses GPU hardware, which is much faster for math like this.  That said, there are a ton of tricks (involving memory hierarchies, cache blocking, and other tricks) that make matrix multiplications go fast on CPUs and other hardware.

For example, try using TensorFlow on the CPU to do the same computation as above:

In [ ]:
withDevice(.cpu) {
    timeMatmulTensor(size: 1)     // Tiny
    timeMatmulTensor(size: 10)    // Bigger
    timeMatmulTensor(size: 100)   // Even Bigger
    timeMatmulTensor(size: 1000)  // Biggerest
    timeMatmulTensor(size: 5000)  // Even Biggerest
}


1x1:
  ⏰average: 0.037117899999999995 ms,   min: 0.033852 ms,   max: 0.045559 ms

10x10:
  ⏰average: 0.0428332 ms,   min: 0.038875 ms,   max: 0.057927 ms

100x100:
  ⏰average: 0.11528369999999999 ms,   min: 0.087688 ms,   max: 0.148416 ms

1000x1000:
  ⏰average: 6.723326900000001 ms,   min: 5.79201 ms,   max: 7.207141 ms

5000x5000:
  ⏰average: 828.9578357999999 ms,   min: 751.708676 ms,   max: 856.148943 ms


This is a pretty big difference.  On my hardware, it takes 2287ms for Swift to do a 1000x1000 multiply on the CPU, it takes TensorFlow 6.7ms to do the same work on the CPU, and takes TensorFlow 0.49ms to do it on a GPU.

# Accelerators vs Flexibility

One of the big challenges with machine learning frameworks today is that they provide a fixed set of "ops" that you can use with high performance.  There is a lot of work underway to fix this.  The [XLA compiler in TensorFlow](https://www.tensorflow.org/xla) is an important piece of this, which allows more flexibility in the programming model while still providing high performance by using compilers to target the hardware accelerator.  If you're interested 

There are other efforts as well, and one of the major things we're working on is called MLIR.



Halide video snippet here - Jeremy.

### Export

In [ ]:
notebookToScript(fname: Path.cwd / "01_matmul.ipynb")